In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
 
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
# Download the files based on its file ID.
# File-1
 
file_id1 = '1Z8lnKok5ufjVdBA-_f1UDmTkJoLqncqY'
module1 = drive.CreateFile({'id': file_id1})
module1.GetContentFile('datasets.py')
 
# File-2
file_id2 = '1wYVBj0dd7QcSfE4IH7Hk1rBZC562k_Ne'
module2 = drive.CreateFile({'id': file_id2})
module2.GetContentFile('deeplabv3.py')
 
# File-3
file_id3 = '1RB063J7lU1HZr8LVW55IKmw48Co_xBCO'
module3 = drive.CreateFile({'id': file_id3})
module3.GetContentFile('utils.py')
 
# File-4
file_id4 = '1_VxRsNJzUOYanJ48JgwcaWoJLtZSL85-'
module4 = drive.CreateFile({'id': file_id4})
module4.GetContentFile('resnet.py')
 
# File-5
file_id5 = '15gIBgRkR0DhdvxtTr7OMyGOiJ6W639IA'
module5 = drive.CreateFile({'id': file_id5})
module5.GetContentFile("aspp.py")
 
 
print('Python files loaded')

Mounted at /content/gdrive
Python files loaded


In [ ]:
import sys
import pandas
import torchvision
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models


import numpy as np
import cv2
import os
from google.colab import drive
import gc
 
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch')
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/model')
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/utils')
 
import resnet
from resnet import ResNet18_OS16, ResNet34_OS16, ResNet50_OS16, ResNet101_OS16, ResNet152_OS16, ResNet18_OS8, ResNet34_OS8
from datasets import DatasetTrain, DatasetVal
from utils import add_weight_decay, MyModel
from deeplabv3 import DeepLabV3
 
 

import time
 
print('Required Libraries Imported')

Required Libraries Imported


In [ ]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf /content/gdrive/My\ Drive/yolov3/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


In [ ]:
# NOTE! NOTE! change this to not overwrite all log data when you train the model:
model_id = "4"

num_epochs = 300
batch_size = 8
learning_rate = 0.0001
base_lr=0.0001/500
max_lr=0.0001
cuda = True if torch.cuda.is_available() else False

print('Hyperparameters set')


Hyperparameters set


In [ ]:
proj_dir = "/content/gdrive/My Drive/deeplabv3/pytorch/"

network = DeepLabV3(model_id, project_dir = proj_dir,num_classes=8)

# train_dataset = DatasetTrain(data_path = proj_dir + "data/train/",
#                              label_path = proj_dir + "data/labels/",enabletransform=True)
# val_dataset = DatasetVal(data_path = proj_dir + "data/val/",
#                          label_path = proj_dir + "data/labels/")


# num_train_batches = int(len(train_dataset)/batch_size)
# num_val_batches = int(len(val_dataset)/batch_size)
# print ("num_train_batches:", num_train_batches)
# print ("num_val_batches:", num_val_batches)

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=batch_size, shuffle=True,
#                                            num_workers=2,pin_memory=True, drop_last=True)
# val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
#                                          batch_size=batch_size, shuffle=False,
#                                          num_workers=2,pin_memory=True, drop_last=True)
# l=len(os.listdir(proj_dir+"training_logs/model_"+model_id+"/checkpoints"))

# #print(os.listdir(proj_dir+"training_logs/model_"+model_id+"/checkpoints")[-1])
# params = add_weight_decay(network, l2_value=0.0001)
# optimizer = torch.optim.Adam(params, lr=learning_rate)
# # Code for Learning rate schedulers 
# #scheduler=torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr, max_lr, step_size_up=320, step_size_down=None, mode='triangular2', gamma=1.0, scale_fn=None, scale_mode='cycle', cycle_momentum=False, base_momentum=0.8, max_momentum=0.9, last_epoch=-1, verbose=False)

# #Random code that has been in the notebook from the start 
# #with open("/content/gdrive/My Drive/StradaImaging/deeplabv3_pytorch/data/cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)
# #    class_weights = np.array(pickle.load(file))
# #class_weights = torch.from_numpy(class_weights)
# #class_weights = Variable(class_weights.type(torch.FloatTensor))#.cuda()

# # loss function
# loss_fn = nn.CrossEntropyLoss()#weight=class_weights)
# #loss_fn = nn.MSELoss()


pretrained resnet, 18<3


In [ ]:
new_weights_path = proj_dir+"training_logs/model_"+model_id+"/checkpoints/"+os.listdir(proj_dir+"training_logs/model_"+model_id+"/checkpoints")[-1] # start training from last saved weights
new_state_dict=torch.load(new_weights_path,map_location="cuda")
city_weights=proj_dir+"pretrained_models/model_13_2_2_2_epoch_580.pth" # start training from orignal pretrained model
city_state_dict=torch.load(city_weights,map_location="cuda")
network = MyModel(network, model_id, proj_dir)
#network.cuda()
network.load_state_dict(new_state_dict)

print(new_weights_path)
device = torch.device("cuda")
network.to(device)

# re define model


# Define tensor types
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


/content/gdrive/My Drive/deeplabv3/pytorch/training_logs/model_4/checkpoints/model_4_epoch_230.pth


In [ ]:
def label_img_to_color(img):
    img=img.astype(np.uint8)
    lut=np.ones((256,1,3),dtype=np.uint8)
    
    ine=np.array([[[0,0,0],#background 0
                   [255,0,0],#signboard blue 1 
                   [0,255,0],#siderail green  2
                   [0,0,255],#sidewall red 3
                   [49,147,245],#traffic cone orange 4
                   [255,255,0],#markerposts cyan 5
                   [255,0,255],#trafficbarrier magenta 6
                   [0,0,255],#manhole yellow 7
                   [255,255, 255],#road white 8
                   [ 79,79,47],#sidewalk gray 9
                   [0,128,128],#pole slate olive 10 
                   [19, 69, 139],]])#fence brown 11
    
    lut[0:12,:,:]=ine.reshape(12,1,3)
    img_color=cv2.applyColorMap(img,lut)

    return img_color

In [ ]:
from google.colab.patches import cv2_imshow # Debugging Import

dir=proj_dir+"data/val/"
paths=os.listdir(dir)
for i,img_path in enumerate(paths):
        
        #Defining the 8 class network  
        
        network = DeepLabV3(model_id, project_dir = proj_dir,num_classes=8)
        network = MyModel(network, model_id, proj_dir)
        network.cuda()
        network.load_state_dict(new_state_dict)
        device = torch.device("cuda")                 #change device on which to perform computations replace cuda with cpu if you want to run on that
        network.to(device)
        Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
        LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor
        
        network.eval() #Set to Inference i.e evaluation or detection mode

        with torch.no_grad(): # (This is done to reduce memory consumption)
       
          print(img_path)
          
          #Preprocess the image to resemble the transformation done while loading the images in datasetloafer function( Datasets.py File )
         
          img = cv2.imread(dir+img_path, -1) # (shape: (1024, 2048, 3))
          oimg = cv2.resize(img, (1024,512), interpolation=cv2.INTER_NEAREST)
          img = oimg/255.0
          img = img - np.array([0.485, 0.456, 0.406])
          img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
          img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
          img = img.astype(np.float32)
          img = torch.from_numpy(img)
          img_batch=torch.unsqueeze(img,0)
          img_batch = Variable(img_batch).cuda()
         
          #Perform inference on 8 class network
         
          output1 = network(img_batch) # (shape: (batch_size, num_classes, img_h, img_w))
          
          
          _, predicted = torch.max(output1.data, 1)
          output1 = output1.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
          pred_label_imgs = np.argmax(output1, axis=1) # (shape: (batch_size, img_h, img_w))
          pred_label_imgs = pred_label_imgs.astype(np.uint8)
          pred1 = pred_label_imgs[0].astype(np.uint8)        

          #Define the second 20 class network
          
          network = DeepLabV3(model_id, project_dir = proj_dir,num_classes=20)
          network.load_state_dict(city_state_dict) 
          network.cuda()
          network.eval()
          output2=network(img_batch)
            
          _, predicted = torch.max(output2.data, 1)
          output2=output2.data.cpu().numpy()
          PRED= np.argmax(output2,axis=1)
          PRED= PRED.astype(np.uint8)    
          pred2=PRED[0]

          # Applying Threshold to detect gray area and create a mask 
          lower_gray = np.array([150,0,20])
          upper_gray = np.array([180,20,255])
          hsv=cv2.cvtColor(oimg,cv2.COLOR_BGR2HSV)
          mask=cv2.inRange(hsv, lower_gray, upper_gray)
          
          #Create a mask that selects only lower third portion of the image
          
          h,l=pred1.shape[0],pred1.shape[1]
          a_mask=np.zeros((pred1.shape),dtype=bool)
          a_mask[h-int(h/3):h]=True
          mask=np.bitwise_and(a_mask,mask)
          # Merge labels from cityscapes onto our 8 class network to produce 12 class labelss
        
          pred1=np.where(np.bitwise_and(pred2>6 ,mask),8,pred1)
          pred1=np.where(pred2==0,8,pred1)
          pred1=np.where(pred2==1,9,pred1)
          pred1=np.where(pred2==5,10,pred1)
          pred1=np.where(pred2==4,11,pred1)
          cv2.imwrite(proj_dir+"/data/Combined/"+img_path[:-3]+"png",pred1)
          net_img=label_img_to_color(pred1)
          city_img=label_img_to_color(pred2)
          city_img=city_img*0.35+oimg*0.65
          net_img=net_img*0.35+oimg*0.65
          total=np.concatenate((city_img,net_img),1)
          total=cv2.resize(total,(1024,512),interpolation=cv2.INTER_NEAREST)    

pretrained resnet, 18<3
M1 J23a Slips_Forward_1048.jpg


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details

pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1054.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1060.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1068.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1132.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1142.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1144.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1150.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1174.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1216.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1218.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1220.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J23a Slips_Forward_1236.jpg
pretrained resnet, 18<3
pretrained resnet, 18<3
M1 J

In [ ]:
import shutil
import random 
copy_path=proj_dir+"data/val/"
other=proj_dir+"data/train/"
labels=proj_dir+"data/Combined/"
paths=random.sample(os.listdir(dir),100)
for img in paths:
  oimg=cv2.imread(dir+img)
  oimg=cv2.resize(oimg,(1024,512),interpolation=cv2.INTER_NEAREST)
  label=cv2.imread(labels+img[:-3]+"png")
  print(labels+img)
  net_img=label_img_to_color(label)
  net_img=net_img*0.35+oimg*0.65
  cv2_imshow(net_img)
  

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

import sys                                          # System bindings
import cv2                                          # OpenCV bindings
import numpy as np


class ColorAnalyser():
    def __init__(self, imageLoc):
        
        img= cv2.imread(imageLoc, 1)
        cv2_imshow(img)
        blur= cv2.medianBlur(img,255)
        cv2_imshow(blur)
        self.src=blur#cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
        
               # Reads in image source
        # Empty dictionary container to hold the colour frequencies
        self.colors_count = {}
        self.height=img.shape[0]
        self.width=img.shape[1]
        self.area=self.height*self.width
    def count_colors(self):
        # Splits image Mat into 3 color channels in individual 2D arrays
        (channel_b, channel_g, channel_r) = cv2.split(self.src)

        # Flattens the 2D single channel array so as to make it easier to iterate over it
        channel_b = channel_b.flatten()
        channel_g = channel_g.flatten()  # ""
        channel_r = channel_r.flatten()  # ""

        for i in range(len(channel_b)):
            HSV = "(" + str(channel_r[i]) + "," + \
                str(channel_g[i]) + "," + str(channel_b[i]) + ")"
            #print(channel_b[i])
            if HSV in self.colors_count:
                self.colors_count[HSV] += 1
            else:
                self.colors_count[HSV] = 1

        print("Colours counted")

    def show_colors(self):
        # Sorts dictionary by value
        print("Top 2colors percentages")
        for i,keys in enumerate(sorted(self.colors_count, key=self.colors_count.__getitem__)):
            # Prints 'key: value'
            if (len(self.colors_count)-i)<3:
              
              print(keys, ": ", (self.colors_count[keys]/self.area)*100)

    def main(self):
        # Checks if an image was actually loaded and errors if it wasn't
        if (self.src is None):
            print("No image data. Check image location for typos")
        else:
            # Counts the amount of instances of HSV values within the image
            self.count_colors()
            # Sorts and shows the colors ordered from least to most often occurance
            self.show_colors()
            # Waits for kegiven as cli argument
Analyser = ColorAnalyser("/content/gdrive/MyDrive/deeplabv3/annotation_images/M1 J23a Slips_Forward_1001.jpg")
Analyser.main()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img=cv2.imread("/content/gdrive/MyDrive/deeplabv3/annotation_images/M1 J23a Slips_Forward_1001.jpg")
#(b,g,r)=cv2.split(img)
hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
#print(b.shape)
# c=1
# print(np.unique(im))
lower_gray = np.array([0,0,20])
upper_gray = np.array([180,20,200])
slices=cv2.inRange(hsv,lower_gray,upper_gray)
# slices=np.zeros(img.shape)
# bg = np.bitwise_and(((im[:,:,0]-im[:,:,1])<c),((im[:,:,0]-im[:,:,1])>(-1*c)),dtype=np.uint8) # B == G
# gr = np.bitwise_and(((im[:,:,1]-im[:,:,2])<c),((im[:,:,1]-im[:,:,2])>(-1*c)),dtype=np.uint8) 
# slices[:,:,0] = np.bitwise_and(bg, gr, dtype= np.uint8)*  img[:,:,0]
# slices[:,:,1] = np.bitwise_and(bg, gr, dtype= np.uint8)*  img[:,:,1]
# slices[:,:,2] = np.bitwise_and(bg, gr, dtype= np.uint8)*  img[:,:,2]
# gray=(np.isclose(b>100,g<150,c))#&np.isclose(g,r,c)&np.isclose(b,r,c))
# gray=np.reshape(gray,(b.shape))
# print(gray.shape)
#cv2_imshow(img)
cv2_imshow(cv2.bitwise_and(img,img, mask= slices))

Output hidden; open in https://colab.research.google.com to view.